In [1]:
import numpy as np
import os
import skimage.io as skio
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
indir = "G:/.shortcut-targets-by-id/1w3kIfvkBH_Xa1G856qbgk-8u0IRdcwZ3/MUSA 650 Final Project"

# set the filepath
filtered_patches_path = os.path.join(indir, "final_patches_w_augmentation")

In [2]:
patches = np.load(os.path.join(filtered_patches_path, "final_patches_w_augmentation.npy"))

patches.shape

(25000, 256, 256, 3)

In [4]:
import tensorflow as tf

# check if GPU is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
import numpy as np
import tensorflow as tf
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [7]:
# Split the dataset into 70% train, 15% validation, and 15% test
train_data, test_data = train_test_split(patches, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [9]:
def create_autoencoder():
    input_img = Input(shape=(256, 256, 3))

    # Encoding layers
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

    # Decoding layers
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

In [10]:
autoencoder = create_autoencoder()
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(train_data, train_data,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(val_data, val_data))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-5].output)
encoded_patches = encoder.predict(patches)

In [ ]:
n_clusters = 5  # Number of clusters to segment the images
encoded_patches_flat = np.reshape(encoded_patches, (len(encoded_patches), -1))
kmeans = KMeans(n_clusters=n_clusters)
cluster_assignments = kmeans.fit_predict(encoded_patches_flat)

In [ ]:
import matplotlib.pyplot as plt

def visualize_cluster(cluster_idx):
    indices = np.where(cluster_assignments == cluster_idx)[0]
    num_images = min(5, len(indices))

    fig, axes = plt.subplots(1, num_images, figsize=(5*num_images, 5))

    for i in range(num_images):
        img = patches[indices[i]]
        axes[i].imshow(img)
        axes[i].axis('off')

    plt.show()

for i in range(n_clusters):
    print(f"Cluster {i+1}:")
    visualize_cluster(i)
